## Hospital General Information Dataset

### By: Anurag Bolneni

This notebook has our preliminary hospital reccomendation system MVP based on the Hospital General Information dataset from CMS. It is split into two sections:
+ Step 1: Data Cleaning & Manipulation
+ Step 2: Taking User Input
+ Step 3: Reccomendation System MVP

## Step 1: Data Cleaning & Manipulation

We first import necessary libraries and use an API to pull data from CMS websites. Then, data is fed into a pandas dataframe where we clean for parameters of interest and data types of different columns.

In [82]:
import pandas as pd
from collections import Counter
import numpy as np

In [2]:
pd.read_csv('Data/Hospital_General_Information.csv').head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Phone Number,Hospital Type,Hospital Ownership,...,Count of READM Measures Better,Count of READM Measures No Different,Count of READM Measures Worse,READM Group Footnote,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,Pt Exp Group Footnote,TE Group Measure Count,Count of Facility TE Measures,TE Group Footnote
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,...,1,9,1,NaN,8,8,NaN,14,11,NaN
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,Acute Care Hospitals,Government - Hospital District or Authority,...,0,9,1,NaN,8,8,NaN,14,14,NaN
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 768-8400,Acute Care Hospitals,Proprietary,...,1,7,1,NaN,8,8,NaN,14,11,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private,...,0,6,0,NaN,8,8,NaN,14,7,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,Acute Care Hospitals,Proprietary,...,0,4,0,NaN,8,Not Available,5.0,14,8,NaN


In [3]:
df_hosp_gen_info = pd.read_csv('Data/Hospital_General_Information.csv').iloc[:,:13].drop(columns=['Phone Number','Meets criteria for promoting interoperability of EHRs'])
df_hosp_gen_info.head()

,Facility ID,Facility Name,Address,City,State,ZIP Code,County Name,Hospital Type,Hospital Ownership,Emergency Services,Hospital overall rating
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,Acute Care Hospitals,Government - Hospital District or Authority,Yes,3
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,Acute Care Hospitals,Government - Hospital District or Authority,Yes,2
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,Acute Care Hospitals,Proprietary,Yes,2
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,Acute Care Hospitals,Voluntary non-profit - Private,Yes,2
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,Acute Care Hospitals,Proprietary,Yes,2


In [4]:
print('Length of unique Facility ID vs total:',len(df_hosp_gen_info['Facility ID'].unique()),',',len(df_hosp_gen_info['Facility ID']))
print('Length of unique address vs total:',len(df_hosp_gen_info.Address.unique()),',',len(df_hosp_gen_info.Address))
print('Variations in hospital overall ratings:', Counter(df_hosp_gen_info['Hospital overall rating']))
print('Emerg_services:', Counter(df_hosp_gen_info['Emergency Services']))
df_hosp_gen_info.dtypes

Length of unique Facility ID vs total: 5306 , 5306
Length of unique address vs total: 5276 , 5306
Variations in hospital overall ratings: Counter({'Not Available': 1996, '3': 1006, '4': 979, '2': 682, '5': 452, '1': 191})
Emerg_services: Counter({'Yes': 4455, 'No': 851})


Facility ID                object
Facility Name              object
Address                    object
City                       object
State                      object
ZIP Code                    int64
County Name                object
Hospital Type              object
Hospital Ownership         object
Emergency Services         object
Hospital overall rating    object
dtype: object

-- Anurag to write blurb about above findings later

We'll be taking the cosine similarity of State and Emergency Services to test our MVP of the reccomendation algorithm. As a last step of cleaning, we need to change Emergency Services (Y/N) to binary classifications.

In [6]:
df_hosp_gen_info['Emergency Services'] = [1 if x=='Yes' else 0 for x in df_hosp_gen_info['Emergency Services']]

In [119]:
df_HCAHPS = pd.read_csv('Data/HCAHPS-Hospital.csv')
df_HCAHPS = df_HCAHPS[['Facility ID','HCAHPS Measure ID','HCAHPS Question','HCAHPS Answer Percent']]
df_HCAHPS['HCAHPS Answer Percent'] = pd.to_numeric(df_HCAHPS['HCAHPS Answer Percent'], errors='coerce')
df_HCAHPS = df_HCAHPS.dropna(axis=0)

df_info = df_HCAHPS.groupby(['Facility ID']).count()
df = df_info[df_info['HCAHPS Question'] == 72].reset_index()
VALID_FACILITY_IDS = list(df['Facility ID'])

df_HCAHPS = df_HCAHPS[df_HCAHPS['Facility ID'].isin(VALID_FACILITY_IDS) == True]

In [120]:
NURSES_MEASURES = ['H_COMP_1_A_P', 'H_NURSE_RESPECT_A_P', 'H_NURSE_LISTEN_A_P', 'H_NURSE_EXPLAIN_A_P']
DOCTORS_MEASURES = ['H_COMP_2_A_P','H_DOCTOR_RESPECT_A_P','H_DOCTOR_LISTEN_A_P', 'H_DOCTOR_EXPLAIN_A_P']
PATIENT_MEASURES = ['H_COMP_3_A_P', 'H_CALL_BUTTON_A_P', 'H_BATH_HELP_A_P']
STAFF_MEASURES = ['H_COMP_5_A_P', 'H_MED_FOR_A_P', 'H_SIDE_EFFECTS_A_P']
#Note, I stopped assessing measures on page 3, but buckets can be further stratified

In [117]:
def NM(df1, df2):
    return df2.loc([((df1['Facility ID']==df2['Facility ID']) & (df1['HCAHPS Measure ID']==measures))['HCAHPS Answer Percent'] for measures in NURSES_MEASURES],'NM')

In [104]:
%%time

hospital_doc_matrix = []

for ids in VALID_FACILITY_IDS[:100]:
    NM = [df_HCAHPS[(df_HCAHPS['Facility ID']==ids) & (df_HCAHPS['HCAHPS Measure ID']==measures)]['HCAHPS Answer Percent'] for measures in NURSES_MEASURES]
    DM = [df_HCAHPS[(df_HCAHPS['Facility ID']==ids) & (df_HCAHPS['HCAHPS Measure ID']==measures)]['HCAHPS Answer Percent'] for measures in DOCTORS_MEASURES]
    PM = [df_HCAHPS[(df_HCAHPS['Facility ID']==ids) & (df_HCAHPS['HCAHPS Measure ID']==measures)]['HCAHPS Answer Percent'] for measures in PATIENT_MEASURES]
    SM = [df_HCAHPS[(df_HCAHPS['Facility ID']==ids) & (df_HCAHPS['HCAHPS Measure ID']==measures)]['HCAHPS Answer Percent'] for measures in STAFF_MEASURES]

    hospital_doc_matrix.append([np.mean(factors) for factors in [NM,DM,PM,SM]])
fin = dict(zip(VALID_FACILITY_IDS[:100],hospital_doc_matrix))

CPU times: user 27.9 s, sys: 4 ms, total: 27.9 s
Wall time: 27.9 s


In [105]:
fin

{'010001': [77.0, 80.75, 60.666666666666664, 63.666666666666664],
 '010005': [75.0, 84.0, 47.666666666666664, 56.0],
 '010006': [69.0, 77.0, 44.666666666666664, 55.333333333333336],
 '010007': [84.0, 85.75, 72.66666666666667, 77.33333333333333],
 '010011': [77.25, 79.25, 65.33333333333333, 63.0],
 '010012': [77.0, 78.5, 60.666666666666664, 51.0],
 '010016': [76.25, 80.0, 58.333333333333336, 62.0],
 '010019': [85.25, 87.75, 65.66666666666667, 60.333333333333336],
 '010021': [79.75, 79.75, 64.33333333333333, 67.33333333333333],
 '010023': [78.75, 81.0, 67.66666666666667, 66.0],
 '010024': [75.75, 78.0, 62.666666666666664, 58.666666666666664],
 '010029': [80.0, 81.25, 67.33333333333333, 60.333333333333336],
 '010033': [79.75, 83.0, 63.666666666666664, 64.66666666666667],
 '010034': [81.75, 79.5, 65.66666666666667, 86.66666666666667],
 '010035': [80.25, 80.25, 60.0, 63.333333333333336],
 '010036': [73.75, 77.5, 62.333333333333336, 58.333333333333336],
 '010038': [79.5, 73.75, 66.0, 58.3333

## Step 2: Take User Input

In [7]:
columns = ['Desired Hospital Rating','Emergency Services','State']
df = pd.DataFrame(columns = columns,dtype=object)

def user_input(df):
    # Take user input for a series of factors
    return df.append({
        'Desired Hospital Rating': int(input('Please rate your desired hospital on scale of 1-5:   ')),
        'Emergency Services': int(input('Do you need emergency services? (Yes = 1, No = 0)     ')),
        'State':str(input('Which State do you live in?     '))},
    ignore_index = True)

In [8]:
user_input_df = user_input(df)

Please rate your desired hospital on scale of 1-5:   5
Do you need emergency services? (Yes = 1, No = 0)     0
Which State do you live in?     MI


## Step 3: Reccommendar System MVP

Our hospital reccomendation system utilizes the cleaned dataset from Step 1 and the User Input from Step 2. We use cosine similarity to determine the top hospitals based on the users target inputs. This section is broken into a set of function that to conduct vectorization of string parmaters as well as numeric parameters to determine cosine similarity of hospitals relative to a user's need. 

For our MVP, we tested our recommender sytem on Emergency Services (Y/N) and State of hospital as initial inputs. Finally, we sort the outcomes by cosine to determine the list of best hospitals. The outcomes so far look like it works relatively well, but we'll need to see how this fluctuates with additional parameters and available data.

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
def give_me_hospitals(df_hospital, df_patient):
    State_List = [state for state in df_hospital.State]
    tfidf_vectorizer = TfidfVectorizer()
    
    sparse_matrix = tfidf_vectorizer.fit_transform(State_List)
    doc_term_matrix = sparse_matrix.toarray()
    X_final = np.array([np.append(doc_term_matrix[i],df_hospital['Emergency Services'][i]) for i in range(len(doc_term_matrix))])

    user_state = df_patient['State']
    Y_transform = tfidf_vectorizer.transform(user_state).toarray()
    Y_final = np.array([np.append(Y_transform, df_patient['Emergency Services'])])

    Cos = cosine_similarity(X_final,Y_final)
    df_hospital['Cosine Similarity'] = [values[0] for values in Cos]
    Recc_df = df_hospital[['Facility Name', 'State', 'Emergency Services','Cosine Similarity']].copy()
    
    return Recc_df.sort_values('Cosine Similarity', ascending=False).reset_index()

In [11]:
df_hospital = df_hosp_gen_info
df_patient = user_input_df
give_me_hospitals(df_hospital, df_patient)

,index,Facility Name,State,Emergency Services,Cosine Similarity
0,2348,OAKLAWN HOSPITAL,MI,0,1.0
1,2375,BARAGA COUNTY MEMORIAL HOSPITAL,MI,0,1.0
2,2369,PAUL OLIVER MEMORIAL HOSPITAL,MI,0,1.0
3,2370,KALKASKA MEMORIAL HEALTH CENTER,MI,0,1.0
4,2371,SCHOOLCRAFT MEMORIAL HOSPITAL,MI,0,1.0
...,...,...,...,...,...
5301,1720,OLATHE MEDICAL CENTER,KS,0,0.0
5302,1719,UNIVERSITY OF KANSAS HOSPITAL,KS,0,0.0
5303,1718,Irwin ACH (FT Riley),KS,0,0.0
5304,1717,PRATT REGIONAL MEDICAL CENTER,KS,0,0.0
